BASE RELATIONNELLE

In [3]:
import pandas as pd
import numpy as np
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display
%load_ext sql
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = 50
%sql sqlite:///trustscore.db



In [3]:
#Infos d'entreprises

df_enterprises = pd.read_csv(r"C:\Users\MSI KATANA B13V\Desktop\PARCOURS DE\Projet Fil Rouge\de_satisfaction_client\data\informations_entreprises.csv", sep=',')
df_enterprises.head()

,review,town,country,institution_type,five_star_%,company_name,trust_score
0,327,Portland,United States,Financial Institution,96%,Evergreen Credit Union,4.9
1,290,Lincoln,United States,Financial Institution,92%,Liberty First Credit Union,4.7
2,3242,Irvine,United States,Financial Institution,66%,GoHenry by Acorns,4.3
3,175,Chicago,United States,Alternative Financial Service,60%,Crypto Dispensers,3.9
4,10,San Francisco,United States,Financial Institution,100%,Swadesh,4.4


In [4]:
#Infos commentaires

df_comments = pd.read_json(r"C:\Users\MSI KATANA B13V\Desktop\PARCOURS DE\Projet Fil Rouge\de_satisfaction_client\data\commentaires.json")
df_comments.head()

,company_name,User,localisation,Titre,commentaire,nombre_reviews,date_experience,reply
0,Evergreen Credit Union,NA,US,I requested bank statements and the…,I requested bank statements and the request wa...,1,"June 18, 2024",None
1,Evergreen Credit Union,NA,US,Great experience,I had multiple scam charges ending in a negati...,1,"June 18, 2024",None
2,Evergreen Credit Union,NA,US,All my questions were answered in a…,All my questions were answered in a timely man...,1,"June 07, 2024",None
3,Evergreen Credit Union,NA,US,Easy and pleasant to deal with,"Easy and pleasant to deal with, understanding ...",1,"June 10, 2024",None
4,Evergreen Credit Union,NA,US,Best Ever Banking Experience,Casey has been amazing to work with. I have be...,1,"May 23, 2024","{'reply_text': 'Hi Lindsay, We're glad Casey ..."


In [12]:
import sqlite3

# Connexion à la base de données SQLite
conn = sqlite3.connect('trustscore.db')
cursor = conn.cursor()

# Création de la table `companies` pour les informations sur les entreprises
cursor.execute('''
CREATE TABLE IF NOT EXISTS companies (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    company_name TEXT,
    town TEXT,
    country TEXT,
    institution_type TEXT
)
''')

# Création de la table `reviews` pour les évaluations des entreprises
cursor.execute('''
CREATE TABLE IF NOT EXISTS reviews (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    company_id INTEGER,
    review INTEGER,
    trust_score REAL,
    five_star_percentage REAL,
    FOREIGN KEY (company_id) REFERENCES companies (id)
)
''')

# Fermer la connexion
conn.commit()
conn.close()



NETTOAYGE ET INSERTION DES DONNEES

Entreprises

In [15]:
import pandas as pd
import sqlite3


# Supposons que votre DataFrame df contient déjà les données nécessaires
df_enterprises['five_star_percentage'] = df_enterprises['five_star_%'].str.rstrip('%').astype(float) / 100
df_enterprises
# Séparation des données des entreprises
df_companies = df_enterprises[['company_name', 'town', 'country', 'institution_type']]

# Connexion à la base de données SQLite
conn = sqlite3.connect('trustscore.db')

# Insérer les données dans la table `companies`
df_companies.to_sql('companies', conn, if_exists='append', index=False, method='multi')

# Récupérer les IDs pour les utiliser dans la table `reviews`
df_companies['id'] = pd.read_sql_query("SELECT id FROM companies", conn)

# Fermer la connexion pour la table des entreprises
conn.close()


C:\Users\MSI KATANA B13V\AppData\Local\Temp\ipykernel_11224\3431661131.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_companies['id'] = pd.read_sql_query("SELECT id FROM companies", conn)


In [17]:
%%sql 

select * from companies limit 5; 

id,company_name,town,country,institution_type
1,Evergreen Credit Union,Portland,United States,Financial Institution
2,Liberty First Credit Union,Lincoln,United States,Financial Institution
3,GoHenry by Acorns,Irvine,United States,Financial Institution
4,Crypto Dispensers,Chicago,United States,Alternative Financial Service
5,Swadesh,San Francisco,United States,Financial Institution


Reviews

In [18]:
# Ajout de la colonne `id` des entreprises à df
df = df_enterprises.join(df_companies.set_index('company_name')['id'], on='company_name')

# Préparation des données des évaluations
df_reviews = df[['id', 'review', 'trust_score', 'five_star_percentage']]
df_reviews.columns = ['company_id', 'review', 'trust_score', 'five_star_percentage']

# Ouverture de la connexion à la base de données
conn = sqlite3.connect('trustscore.db')

# Insérer les données dans la table `reviews`
df_reviews.to_sql('reviews', conn, if_exists='append', index=False, method='multi')

# Fermer la connexion finale
conn.close()


In [19]:
%%sql

select *

from reviews; 



id,company_id,review,trust_score,five_star_percentage
1,1,327,4.9,0.96
2,2,290,4.7,0.92
3,3,3242,4.3,0.66
4,4,175,3.9,0.6
5,5,10,4.4,1.0
6,6,4,4.0,1.0
7,7,9,3.8,0.67
8,8,1,3.7,1.0
9,9,1,3.7,1.0
10,10,1,3.7,1.0


In [8]:
%%sql

select avg(review) as _avg_review,
       town,
       company_name
from companies
GROUP BY town,company_name; 

_avg_review,town,company_name
1.0,None,ASDFTEST
0.0,None,Covault
12.0,None,Heritage Valley Federal Credit Union
3.0,None,cryptobaseatm.com
0.0,Bronx,Slide2Thrive
0.0,Carlsbad,ATMDepot.com
175.14285714285714,Chicago,Crypto Dispensers
0.0,Chicago,Liberty Bank for Savings
19.0,Cookeville,Cash Express
0.0,Fort Lauderdale,Nationwide Payment Systems


BASE ORIENTEE DOCUMENT

Etape 1 : "Il faut s'assurer que MongoDB est installé et actif."
Etape 2 : " Lancer la commande docker-compose up -d  qui va initialiser le Docker avec tous les composants nécéssaire voir fichier .yml"

In [1]:
import json
from pymongo import MongoClient

# Chargement du fichier sur les commentaires
json_file = "../data/df_commentaires_par_entreprise.json"

# Connexion à MongoDB (vérifier que Docker est en cours d'exécution)
client = MongoClient('mongodb://localhost:27017/')

# Sélection de la base de données
db = client['trustscore']

# Sélection de la collection (ou création si elle n'existe pas)
collection = db['commentaires']

# Lecture du fichier JSON et insertion des données dans la collection
with open(json_file, 'r', encoding='utf-8') as file:
    data = json.load(file)
    collection.insert_many(data)

# Fermeture de la connexion à MongoDB
client.close()

print("Insertion des données terminée.")


Insertion des données terminée.


CREATION DES INDEX SUR LES COLONNES D'AGGREGATION

In [21]:
from pymongo import MongoClient

# Connexion à MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['trustscore']
collection = db['commentaires']

# Création des index
collection.create_index([('company_name', 1)])
collection.create_index([('date_experience', 1)])
collection.create_index([('localisation', 1)])

print("Indexation terminée.")
client.close()


Indexation terminée.


QUELQUES REQUETES

In [2]:
import json
from pymongo import MongoClient

# Connexion à MongoDB
client = MongoClient('mongodb://localhost:27017/')
comments = client["trustscore"]["commentaires"]

# Exécuter les requêtes avant de fermer la connexion

total_count = comments.count_documents({})
print("Nombre total de documents :", total_count)


results = comments.find({'date_experience': {'$gte': '2024-06-01'}}, {'_id': 0, 'company_name': 1, 'date_experience': 1})
for document in results:
    print(document)



results = comments.find()
for document in results:
    print(document)







Nombre total de documents : 564
{'company_name': 'Evergreen Credit Union', 'date_experience': 'October 15, 2024'}
{'company_name': 'Evergreen Credit Union', 'date_experience': 'September 25, 2024'}
{'company_name': 'Evergreen Credit Union', 'date_experience': 'August 02, 2024'}
{'company_name': 'Evergreen Credit Union', 'date_experience': 'July 02, 2024'}
{'company_name': 'Evergreen Credit Union', 'date_experience': 'July 18, 2024'}
{'company_name': 'Evergreen Credit Union', 'date_experience': 'July 02, 2024'}
{'company_name': 'Evergreen Credit Union', 'date_experience': 'July 09, 2024'}
{'company_name': 'Evergreen Credit Union', 'date_experience': 'April 20, 2024'}
{'company_name': 'Evergreen Credit Union', 'date_experience': 'May 23, 2024'}
{'company_name': 'Evergreen Credit Union', 'date_experience': 'June 18, 2024'}
{'company_name': 'Evergreen Credit Union', 'date_experience': 'November 21, 2023'}
{'company_name': 'Evergreen Credit Union', 'date_experience': 'July 30, 2024'}
{'comp

In [40]:
# Fermeture de la connexion à MongoDB
client.close()

CREATION DE L'INDEX

In [1]:
from elasticsearch import Elasticsearch, helpers
from pymongo import MongoClient
from datetime import datetime

# Connexion à MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['trustscore']
collection = db['commentaires']

# Connexion à Elasticsearch
es = Elasticsearch(['http://localhost:9200'])
if not es.ping():
    print("Connexion à Elasticsearch échouée.")
    exit(1)

# Fonction pour convertir et vérifier les dates
def convert_date(date_str, format_str):
    try:
        return datetime.strptime(date_str, format_str).isoformat()
    except ValueError as e:
        print(f"Erreur de conversion de date '{date_str}': {e}")
        return None

# Fonction pour indexer les données dans Elasticsearch
def index_data():
    cursor = collection.find({}, {'_id': 0})  # Exclure le champ _id
    actions = []
    for document in cursor:
        # Convertir le champ date_experience en un format compatible
        if 'date_experience' in document and document['date_experience']:
            converted_date = convert_date(document['date_experience'], "%B %d, %Y")
            if not converted_date:
                print(f"Document avec une date_experience invalide ignoré: {document}")
                continue  # Skip this document if date conversion fails
            document['date_experience'] = converted_date

        # Convertir le champ reply_date en un format compatible
        if 'reply' in document and document['reply'] and 'reply_date' in document['reply'] and document['reply']['reply_date']:
            converted_reply_date = convert_date(document['reply']['reply_date'], "%A, %B %d, %Y at %I:%M:%S %p")
            if not converted_reply_date:
                print(f"Document avec une reply_date invalide ignoré: {document}")
                continue  # Skip this document if date conversion fails
            document['reply']['reply_date'] = converted_reply_date

        actions.append({
            "_index": "commentaires_index",
            "_source": document
        })

    # Bulk indexation with error logging
    try:
        helpers.bulk(es, actions)
        print("Indexation terminée.")
    except helpers.BulkIndexError as e:
        print(f"Bulk indexing a échoué: {len(e.errors)} documents n'ont pas pu être indexés.")
        for error in e.errors:
            print(f"Erreur pour le document: {error}")

# Appel de la fonction d'indexation
index_data()

# Fermer la connexion à MongoDB
client.close()


C:\Users\MSI KATANA B13V\AppData\Local\Temp\ipykernel_34920\1217541013.py:12: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  if not es.ping():
C:\Users\MSI KATANA B13V\AppData\Local\Temp\ipykernel_34920\1217541013.py:52: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  helpers.bulk(es, actions)


Indexation terminée.


POSTGRES

In [3]:
import psycopg2

# Connexion à la base de données PostgreSQL
conn = psycopg2.connect(
    host="localhost",  # Si ton Docker expose le port local, sinon utilise 'postgres' si dans Docker Compose
    port="5432",       # Le port par défaut de PostgreSQL
    database="trustscore_db",
    user="myuser",
    password="mypassword"
)

# Créer un curseur pour exécuter des requêtes
cursor = conn.cursor()


In [4]:
# Commande SQL pour créer la table avec la structure correspondant à ton CSV
create_table_query = '''
CREATE TABLE IF NOT EXISTS companies (
    company_id SERIAL PRIMARY KEY,
    town VARCHAR(255),
    country VARCHAR(255),
    institution_type VARCHAR(255),
    company_name VARCHAR(255),
    trust_score DECIMAL(3, 2),
    review INTEGER,
    five_star_percentage DECIMAL(5, 2)
);
'''

# Exécuter la requête pour créer la table
cursor.execute(create_table_query)
conn.commit()  # Valider la transaction

print("Table 'companies' créée avec succès.")



Table 'companies' créée avec succès.


In [5]:
import pandas as pd

# Lire le fichier CSV avec pandas
df = pd.read_csv('..\data\informations_entreprises.csv')

# Nettoyer les données - convertir 'five_star_%' en décimal (de 96% à 96.00)
df['five_star_percentage'] = df['five_star_%'].str.replace('%', '').astype(float)

# Insérer les données dans PostgreSQL
for index, row in df.iterrows():
    cursor.execute(
        """
        INSERT INTO companies (town, country, institution_type, company_name, trust_score, review, five_star_percentage)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """,
        (row['town'], row['country'], row['institution_type'], row['company_name'], row['trust_score'], row['review'], row['five_star_percentage'])
    )

conn.commit()  # Valider la transaction
print("Données CSV insérées dans PostgreSQL avec succès.")


Données CSV insérées dans PostgreSQL avec succès.


In [6]:
cursor.execute("SELECT * FROM companies LIMIT 10;")
records = cursor.fetchall()

for row in records:
    print(row)


(25, 'Portland', ' United States', 'Credit Union', 'NaN', Decimal('4.80'), 345, Decimal('95.00'))
(26, 'Lincoln', ' United States', 'ATM', 'NaN', Decimal('4.80'), 316, Decimal('92.00'))
(27, 'Irvine', ' United States', 'Bank', 'NaN', Decimal('4.30'), 3320, Decimal('66.00'))
(28, 'Chicago', ' United States', 'Cryptocurrency Service', 'NaN', Decimal('4.10'), 226, Decimal('64.00'))
(29, 'Cleveland', ' United States', 'Skin Care Products Vending Machine', 'NaN', Decimal('4.50'), 14, Decimal('100.00'))
(30, 'Atlanta', ' United States', 'Financial Institution', 'NaN', Decimal('4.40'), 15, Decimal('93.00'))
(31, 'San Francisco', ' United States', 'Financial Institution', 'NaN', Decimal('4.40'), 10, Decimal('100.00'))
(32, 'San Diego ', ' United States', 'Business to Business Service', 'NaN', Decimal('4.00'), 4, Decimal('100.00'))
(33, 'San Jose', ' United States', 'Debt Relief Service', 'NaN', Decimal('3.80'), 9, Decimal('67.00'))
(34, 'Las Vegas', ' United States', 'ATM', 'NaN', Decimal('3.7

In [ ]:
cursor.execute("TRUNCATE TABLE companies;")

In [24]:
# Fermer le curseur et la connexion
cursor.close()
conn.close()
print("Connexion PostgreSQL fermée.")


Connexion PostgreSQL fermée.


In [7]:
import psycopg2


# Section 2: Chargement des bases de données avec Python
# Connexion à SQLite
#conn = sqlite3.connect('../src/trustscore.db')
#df = pd.read_sql_query("SELECT * FROM companies_infos", conn)
#conn.close()



# Connexion à PostgreSQL
conn = psycopg2.connect(
    host="localhost",  # Remplace par ton hôte PostgreSQL
    port="5432",       # Port par défaut de PostgreSQL
    database="trustscore_db",
    user="myuser",
    password="mypassword"
)

# Exécuter la requête SQL et charger les données dans un DataFrame pandas
df = pd.read_sql_query("SELECT * FROM companies", conn)

# Fermer la connexion
conn.close()


C:\Users\MSI KATANA B13V\AppData\Local\Temp\ipykernel_6836\3315423592.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM companies", conn)


In [25]:
import psycopg2



# Connexion à PostgreSQL
conn = psycopg2.connect(
    host="localhost",  # Remplace par ton hôte PostgreSQL
    port="5432",       # Port par défaut de PostgreSQL
    database="trustscore_db",
    user="myuser",
    password="mypassword"
)

# Exécuter la requête SQL et charger les données dans un DataFrame pandas
df = pd.read_sql_query("SELECT * FROM companies", conn)



C:\Users\MSI KATANA B13V\AppData\Local\Temp\ipykernel_32740\3043954848.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM companies", conn)


In [26]:
print(df.columns)

Index(['company_id', 'town', 'country', 'institution_type', 'company_name',
       'trust_score', 'review', 'five_star_percentage'],
      dtype='object')


In [7]:
# Fermer la connexion
conn.close()